In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset

import matplotlib.pyplot as plt
import pickle
import os
import json

## Dataloader

In [154]:
with open(root_dir + 'index0.json') as f:
    j = torch.Tensor(json.load(f))

In [318]:
clean_dir = '/home/tk/Documents/clean/' 
mix_dir = '/home/tk/Documents/mix/' 
clean_label_dir = '/home/tk/Documents/clean_labels/' 
mix_label_dir = '/home/tk/Documents/mix_labels/' 


class MSourceDataSet(Dataset):
    
    def __init__(self, clean_dir, mix_dir, clean_label_dir, mix_label_dir):
                
        with open(clean_dir + 'clean0.json') as f:
            clean0 = torch.Tensor(json.load(f))
            
        with open(mix_dir + 'mix0.json') as f:
            mix0 = torch.Tensor(json.load(f))
        
        with open(clean_label_dir + 'clean_label0.json') as f:
            clean_label0 = torch.Tensor(json.load(f))
            

        with open(mix_label_dir + 'mix_label0.json') as f:
            mix_label0 = torch.Tensor(json.load(f))
        
        
        self.spec = torch.cat([clean0, mix0], 0)
        self.label = torch.cat([clean_label0, mix_label0], 0)
        
    def __len__(self, a):
        
        if a == "x":
            return len(self.spec)
        if a == "y":
            return len(self.label)
                
    def __getitem__(self, index): 

        spec = self.spec[index]
        label = self.label[index]
        return spec, label

In [319]:
trainset = MSourceDataSet(clean_dir, mix_dir, clean_label_dir, mix_label_dir)

In [327]:
a, b = trainset.__getitem__(1000)
print (b)

tensor([1., 0., 0., 0., 0., 1., 0., 0., 0., 0.])


In [229]:
trainloader = torch.utils.data.DataLoader(dataset = trainset,
                                                batch_size = 4,
                                                shuffle = False)

# testloader = torch.utils.data.DataLoader(dataset = testset,
#                                                batch_size = 4,
#                                                shuffle = True)

In [188]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1025*16, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)

        
    def forward(self, x):
        x = x.view(-1, 1025*16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
model = Net()
print (model)

Net(
  (fc1): Linear(in_features=16400, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [202]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.03)

In [230]:
for i, data in enumerate(trainloader, 0):
    inputs, labels = data

In [231]:
type(labels)

torch.Tensor

In [223]:
labels = labels.astype(np.int32)
labels = torch.FloatTensor(labels)

AttributeError: 'Tensor' object has no attribute 'astype'

In [232]:
model.train()
for epoch in range(10):
    running_loss = 0
    
    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data

        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
#         if i % 2000 == 0:
#             print (loss.item())
        running_loss += loss.item()
        if i % 1000 == 0:
            print ('[%d, %5d] loss: %.3f' % (epoch, i, running_loss/ 1000))
            running_loss = 0
            
torch.save(model, 'FeatureNet.pkl')

RuntimeError: input and target shapes do not match: input [4 x 10], target [4 x 2] at /opt/conda/conda-bld/pytorch_1535493744281/work/aten/src/THNN/generic/MSECriterion.c:12

In [293]:
import torch
a = torch.ones([1,2])
b = torch.ones([1,2])
print (a.shape)

c = torch.cat([a,b],0)
print (c.shape)

torch.Size([1, 2])
torch.Size([2, 2])
